<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日常支出速算與分攤（作業一）
*   目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
*   AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
*   Sheet 欄位：date,time,item,money,category,pay_way
*   googlesheet:https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing





In [78]:
!pip install gspread oauth2client

In [79]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import pandas as pd
import datetime
#from types import new_class
import gradio as gr

creds,_ = default()
gc = gspread.authorize(creds)
gsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing')
worksheet = gsheet.worksheet('工作表1')
TOTAL_CELL = 'G2'

In [80]:
def write_to_sheet(date_str, time_str, item, money, category, pay_way):
    """
    接收使用者輸入，並寫入 Google Sheets 的函式。
    """
    try:
        # 驗證日期和時間格式
        datetime.datetime.strptime(date_str, '%Y-%m-%d')
        datetime.datetime.strptime(time_str, '%H:%M')

        # 確保金額是數字
        money = int(money)

        # 建立一個 DataFrame 來儲存單筆資料
        data = pd.DataFrame([{
            '日期': date_str,
            '時間': time_str,
            '品項': item,
            '金額': money,
            '類別': category,
            '支付方式': pay_way
        }])

        # 將 DataFrame 轉為列表以便寫入
        data_to_write = data.values.tolist()

        # 寫入資料到 Google Sheets
        worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')

        # 寫入成功後，【呼叫更新總計的函式】來更新 Sheet 和 Gradio 介面
        new_total_output = update_total_in_sheet()

        return "✅ 資料已成功寫入！", new_total_output, money


    except ValueError as ve:
        # 處理格式錯誤
        current_total_output = update_total_in_sheet()
        return f"❌ 格式錯誤：{ve}，請確認日期（YYYY-MM-DD）、時間（HH:MM）或金額的格式是否正確。", current_total_output, None
    except Exception as e:
        # 處理其他錯誤
        current_total_output = update_total_in_sheet()
        return f"❌ 寫入失敗：{e}", current_total_output, None


In [81]:
def get_total_amount_value():
    """
    從 Google Sheets 中讀取所有資料，計算金額總計並回傳【標準 Python 數字】。
    """
    try:
        # 讀取工作表的所有記錄（不包含標題行）
        df = pd.DataFrame(worksheet.get_all_records())

        if '金額' in df.columns:
            # 強制將總和轉換為標準 Python float 或 int，解決 int64 的問題
            total_money = float(df['金額'].sum(skipna=True))
            # 如果您確定不會有小數，也可以使用 int(...)，但 float 更安全
            return total_money
        else:
            print("錯誤：找不到 '金額' 欄位。")
            return 0.0 # 使用浮點數 0.0
    except Exception as e:
        print(f"總計計算失敗：{e}")
        return 0.0 # 使用浮點數 0.0

In [82]:
def populate_total_amount(amount_state):
    """
    從 gr.State 中讀取金額並填充到輸入框。
    """
    return amount_state

In [83]:
def update_total_in_sheet():
    """
    計算總計，並將結果寫回 Google Sheets 的指定儲存格 (G1)。
    """
    total_value = get_total_amount_value() # total_value 現在是標準 Python float/int

    # 寫入 Google Sheet
    # 這裡的 total_value 是數字，gspread 可以處理
    worksheet.update_cell(2, 7, total_value) # 1代表第一行，7代表第七欄(G)

    # 回傳【格式化字串】給 Gradio 介面顯示
    # 這裡必須是字串，因為 total_output 是 gr.Markdown
    return f"💰 總支出：{total_value:.2f} 元"

In [84]:
def split_bill(total_amount, num_people):
    """
    分帳計算函式。
    """
    try:
        total_amount = float(total_amount)
        num_people = int(num_people)

        if num_people <= 0:
            return "❌ 人數必須大於 0。"

        per_person_amount = total_amount / num_people
        return f"✅ 每人應付金額：{per_person_amount:.2f} 元"

    except ValueError:
        return "❌ 請確認金額和人數都為數字。"
    except Exception as e:
        return f"❌ 計算失敗：{e}"

In [85]:
with gr.Blocks(title="chi_支出記錄器") as demo:
    gr.Markdown("## 📋 Google Sheets 支出記錄器")

    # 這裡新增一個 gr.State 來儲存金額
    amount_state = gr.State(value=None)

    with gr.Tabs():
        # 第一個分頁：支出記錄器
        with gr.TabItem("💸 支出記錄器"):
            gr.Markdown("請在下方輸入支出資料，資料將會自動寫入 Google Sheets。")

            with gr.Row():
                date_in = gr.Textbox(label="日期", placeholder="YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
                time_in = gr.Textbox(label="時間", placeholder="HH:MM", value=datetime.datetime.now().strftime("%H:%M"))

            with gr.Row():
                item_in = gr.Textbox(label="品項", placeholder="例如：午餐、車票")
                money_in = gr.Textbox(label="金額", placeholder="數字")

            with gr.Row():
                category_in = gr.Textbox(label="類別", placeholder="例如：外食、交通")
                pay_way_in = gr.Textbox(label="支付方式", placeholder="例如：現金、信用卡")

            submit_btn = gr.Button("寫入資料，並計算新總金額")

            with gr.Row():
                output_msg = gr.Markdown()
                # 介面載入時，直接呼叫 update_total_in_sheet() 來顯示當前總計
                total_output = gr.Markdown(value=update_total_in_sheet())

            # submit_btn.click 的設定保持不變，因為 write_to_sheet 已經更新了邏輯
            submit_btn.click(
                fn=write_to_sheet,
                inputs=[date_in, time_in, item_in, money_in, category_in, pay_way_in],
                outputs=[output_msg, total_output, amount_state]
            )

        # 第二個分頁：分帳小工具
        with gr.TabItem("💰 分帳小工具"):
            gr.Markdown("請輸入總金額和人數，按下按鈕即可計算每人應付金額。")

            with gr.Row():
                total_value = gr.Textbox(label="總金額", placeholder="例如：1000")
                num_people_in = gr.Textbox(label="人數", placeholder="例如：4")

            # 新增一個按鈕來從支出頁面取得金額
            fill_btn = gr.Button("讀取最後一筆支出金額") # <--- 這是您原本遺失的按鈕定義
            split_btn = gr.Button("計算分帳")

            split_output = gr.Markdown()

           # 設定點擊事件
            fill_btn.click(
                fn=populate_total_amount,
                inputs=amount_state,
                outputs=total_value # <--- 修正為 total_value (與上面定義的名稱一致)
            )

            split_btn.click(
                fn=split_bill,
                inputs=[total_value, num_people_in], # <--- 修正為 total_value
                outputs=split_output
            )


In [86]:
demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>